In [1]:
from influxdb import DataFrameClient
from datetime import datetime, timedelta
import pandas as pd
import time

In [2]:
#given by example to fetch the data
def query_data(time_range=["",""],rectangle=[0,0,0,0]):
    host='134.76.15.89'
    user = 'ofread'
    password = '9VpCm7wrfgBo'
    dbname = 'openforecast'
    protocol = 'line'
    port=8086

    print(time_range[0])
    client = DataFrameClient(host, port, user, password, dbname, ssl=True, verify_ssl=False)

    bind_params = {
        "start_time_epoch": time_range[0],
        "end_time_epoch": time_range[1]#,
    #    "sensor_id": sensor_id
    }

    #measurement = "P1,P2,temperature,humidity"
    measurement = "temperature,lat,lon,sensor_id"

    query = f"select {measurement} from sensor where \
                    time > $start_time_epoch AND \
                    time < $end_time_epoch AND \
                    lon > {rectangle[0]} AND lon < {rectangle[2]} AND \
                    lat > {rectangle[1]} AND lat < {rectangle[3]}"
    res=client.query(query, bind_params=bind_params)
    if 'sensor' in res.keys():
        return res['sensor']
    else:
        return {}

In [3]:
#takes start date e.g. datetime(2016, 1, 1) and number of days n
#returns days as time_range
def generate_time_ranges(start_date,n):
    for day in range(n):  
        start_time = start_date + timedelta(days=day)
        end_time = start_time + timedelta(days=1)
        
        time_range = [start_time.strftime('%Y-%m-%dT%H:%M:%SZ'), end_time.strftime('%Y-%m-%dT%H:%M:%SZ')]
        yield time_range

In [4]:
#runs queries for a time_range
def process_data(start_date, data, generate_time_ranges, query_data, num_days, rectangle):
    for time_range in generate_time_ranges(start_date, num_days):
        # Query data for the specified time range and rectangle
        df = pd.DataFrame(query_data(time_range, rectangle))
        # Append the aggregated data to the provided data list
        data.append(df)



In [5]:
#Usage example:
data = []
rectangle = [13.3115, 52.4235, 13.5115, 52.6235]
process_data(start_date=datetime(2020, 1, 1), data=data, generate_time_ranges=generate_time_ranges, query_data=query_data, num_days=1, rectangle=rectangle)


2020-01-01T00:00:00Z


/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.76.15.89'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


In [5]:
city_names = [
    "Tirane / Tirana", "Andorra la Vella", "Minsk", "Bruxelles / Brüssel", 
    "Sarajevo", "Sofiya / Sofia", "Kobenhavn / Kopenhagen", "Berlin", 
    "Tallinn", "Helsinki", "Paris", "Athinai / Athen", "London", "Dublin", 
    "Reykjavik", "Roma / Rom", "Prishtine / Pristina", "Zagreb", "Riga", 
    "Vaduz", "Vilnius", "Luxembourg / Luxemburg", "Valletta", "Chisinau", 
    "Monaco", "Podgorica", "Amsterdam", "Skopje", "Oslo", "Vienna / Wien", 
    "Warszawa / Warschau", "Lisboa / Lissabon", "Bucuresti / Bukarest", 
    "Moskva / Moskau", "San Marino", "Stockholm", "Bern", "Beograd / Belgrad", 
    "Bratislava", "Ljubljana", "Madrid", "Praha / Prag", "Kyiv / Kiew", 
    "Budapest"
]

city_coordinates = [
    [41.3317, 19.8172],
    [42.5075, 1.5218],
    [53.9678, 27.5766],
    [50.8371, 4.3676],
    [43.8608, 18.4214],
    [42.7105, 23.3238],
    [55.6763, 12.5681],
    [52.5235, 13.4115],
    [59.4389, 24.7545],
    [60.1699, 24.9384],
    [48.8567, 2.3510],
    [37.9792, 23.7166],
    [51.5002, -0.1262],
    [53.3441, -6.2675],
    [64.1353, -21.8952],
    [41.8955, 12.4823],
    [42.6740, 21.1788],
    [45.8150, 15.9785],
    [56.9465, 24.1049],
    [47.1411, 9.5215],
    [54.6896, 25.2799],
    [49.6100, 6.1296],
    [35.9042, 14.5189],
    [47.0167, 28.8497],
    [43.7325, 7.4189],
    [42.4602, 19.2595],
    [52.3738, 4.8910],
    [42.0024, 21.4361],
    [59.9138, 10.7387],
    [48.2092, 16.3728],
    [52.2297, 21.0122],
    [38.7072, -9.1355],
    [44.4479, 26.0979],
    [55.7558, 37.6176],
    [43.9424, 12.4578],
    [59.3328, 18.0645],
    [46.9480, 7.4481],
    [44.8048, 20.4781],
    [48.2116, 17.1547],
    [46.0514, 14.5060],
    [40.4167, -3.7033],
    [50.0878, 14.4205],
    [50.4422, 30.5367],
    [47.4984, 19.0408]
]

In [6]:
#europe_capitals = [[v[1], v[0]] for v in city_coordinates] #switch coordinates
rectangles = []
for c in city_coordinates:
    rectangles.append([c[1]-0.1, c[0]-0.1, c[1]+0.1, c[0]+0.1])
print(city_names[7], rectangles[7])

Berlin [13.3115, 52.4235, 13.5115, 52.6235]


In [7]:
#main routine 

start_time = time.time()

#Usage example:
data = []
start_date = datetime(2022,10,15)
rectangle = rectangles[7]
num_days = 78
process_data(
    start_date=start_date, 
    data=data, 
    generate_time_ranges=generate_time_ranges, 
    query_data=query_data, 
    num_days=num_days, 
    rectangle=rectangle)

end_time= time.time()
print(end_time-start_time)

2022-10-15T00:00:00Z


/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.76.15.89'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


2022-10-16T00:00:00Z


/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.76.15.89'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


2022-10-17T00:00:00Z


...

2022-12-30T00:00:00Z


/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.76.15.89'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


2022-12-31T00:00:00Z


/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.76.15.89'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


1733.556993484497


In [8]:
len(data)

102

In [9]:
data[-1]

,temperature,lat,lon,sensor_id
2022-10-14 00:00:01+00:00,NaN,52.560000,13.374000,2055
2022-10-14 00:00:01+00:00,NaN,52.477976,13.385483,64413
2022-10-14 00:00:01+00:00,10.40,52.477976,13.385483,64414
2022-10-14 00:00:02+00:00,10.50,52.560000,13.374000,2056
2022-10-14 00:00:03+00:00,10.55,52.560000,13.374000,67015
...,...,...,...,...
2022-10-14 23:59:55+00:00,13.40,52.467103,13.436735,46404
2022-10-14 23:59:55+00:00,NaN,52.467103,13.436735,46403
2022-10-14 23:59:56+00:00,NaN,52.523424,13.402184,74986
2022-10-14 23:59:58+00:00,14.30,52.530000,13.390000,19728


### Taking hourly means per sensors and save

In [8]:
df_list = []
for l in data:
    df_list.append(pd.DataFrame(l))
df = pd.concat(df_list, ignore_index=False)
df = df.dropna(subset=['temperature'])
df['date'] = df.index.date
df['hour'] = df.index.hour

# 2. Group by 'date' and 'sensor_id' and calculate the required statistics
aggregated = df.groupby(['date','hour','sensor_id']).agg(
    temperature_mean=('temperature', 'mean'),
    lat=('lat', 'first'),  # Assuming lat doesn't change for a sensor
    lon=('lon', 'first')   # Assuming lon doesn't change for a sensor
).reset_index()

# 3. Optionally, if you want the 'date' column to be in datetime format
aggregated['date'] = pd.to_datetime(aggregated['date'])
aggregated.to_csv('berlin2022c.csv')

### Appendix: hourly means

In [7]:
#fetch raw example data (1 hour Berlin)
time_range=['2020-01-01T00:00:00Z', '2020-01-01T01:00:00Z']
rectangle=[13.3115, 52.4235, 13.5115, 52.6235]
raw = query_data(time_range,rectangle)
raw

2020-01-01T00:00:00Z


/opt/conda/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host '134.76.15.89'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,temperature,lat,lon,sensor_id
2020-01-01 00:00:01+00:00,6.49,52.582000,13.406000,37450
2020-01-01 00:00:02+00:00,NaN,52.532000,13.436000,4337
2020-01-01 00:00:04+00:00,NaN,52.462000,13.416000,31996
2020-01-01 00:00:04+00:00,NaN,52.564698,13.400098,34174
2020-01-01 00:00:05+00:00,5.20,52.462000,13.416000,31997
...,...,...,...,...
2020-01-01 00:59:57+00:00,NaN,52.496000,13.338000,1444
2020-01-01 00:59:58+00:00,5.30,52.480000,13.350000,9406
2020-01-01 00:59:59+00:00,NaN,52.566000,13.346000,34775
2020-01-01 00:59:59+00:00,6.90,52.556000,13.394000,2116


In [8]:
#load saved data to compare
compare = pd.read_csv("berlin2020.csv")
compare = compare[compare.date == "2020-01-01"]
compare = compare[compare.hour == 0]
compare

,Unnamed: 0.1,Unnamed: 0,date,hour,sensor_id,temperature_mean,lat,lon
0,0,0,2020-01-01,0,10163,4.904167,52.556,13.384
1,1,1,2020-01-01,0,10386,4.488000,52.482,13.360
2,2,2,2020-01-01,0,11958,5.316667,52.576,13.348
3,3,3,2020-01-01,0,11991,5.100000,52.522,13.446
4,4,4,2020-01-01,0,12071,5.782500,52.560,13.378
...,...,...,...,...,...,...,...,...
183,183,183,2020-01-01,0,9393,5.175000,52.508,13.422
184,184,184,2020-01-01,0,9406,5.508000,52.480,13.350
185,185,185,2020-01-01,0,9768,3.791667,52.520,13.494
186,186,186,2020-01-01,0,9772,4.526316,52.588,13.370


In [9]:
print(f"Difference before dropping NaN: {len(raw)-len(compare)}")
raw = raw.dropna(subset=['temperature'])
print(f"Difference after dropping NaN: {len(raw)-len(compare)}")

Difference before dropping NaN: 8706
Difference after dropping NaN: 4180


In [13]:
#number of data points per sensor
unique_sensors = raw["sensor_id"].unique() #find unique sensors (np.array)
len(unique_sensors)
sensor_entry_counts = pd.DataFrame({
    'sensor_id': unique_sensors,
    'entry_count': [len(raw[raw.sensor_id == s]) for s in unique_sensors]
}) #count number of entries for unique sensors (dataframe)
sensor_entry_counts = sensor_entry_counts.sort_values(by='entry_count', ascending=False).reset_index(drop=True) #sort
sensor_entry_counts

,sensor_id,entry_count
0,15845,100
1,15844,100
2,13369,58
3,2130,25
4,19728,25
...,...,...
183,13790,4
184,16473,4
185,13596,3
186,29391,2


In [11]:
#data points for one sensor in one hour 
raw[raw.sensor_id == "15845"]

,temperature,lat,lon,sensor_id
2020-01-01 00:00:18+00:00,10.4,52.498,13.466,15845
2020-01-01 00:00:56+00:00,10.4,52.498,13.466,15845
2020-01-01 00:01:31+00:00,10.4,52.498,13.466,15845
2020-01-01 00:02:07+00:00,10.4,52.498,13.466,15845
2020-01-01 00:02:44+00:00,10.4,52.498,13.466,15845
...,...,...,...,...
2020-01-01 00:57:27+00:00,9.6,52.498,13.466,15845
2020-01-01 00:58:02+00:00,9.6,52.498,13.466,15845
2020-01-01 00:58:37+00:00,9.6,52.498,13.466,15845
2020-01-01 00:59:13+00:00,9.6,52.498,13.466,15845


In [19]:
print(f"Sum of all measurements: {raw[raw.sensor_id == '15845'].temperature.sum()}")
print(f"Mean: {raw[raw.sensor_id == '15845'].temperature.mean()}")

Sum of all measurements: 1007.4000000000001
Mean: 10.074000000000002


In [20]:
raw['date'] = raw.index.date
raw['hour'] = raw.index.hour

aggregated = raw.groupby(['date','hour','sensor_id']).agg(
    temperature_mean=('temperature', 'mean'),
    lat=('lat', 'first'),  # Assuming lat doesn't change for a sensor
    lon=('lon', 'first')   # Assuming lon doesn't change for a sensor
).reset_index()

/tmp/ipykernel_1521/4050028980.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['date'] = raw.index.date
/tmp/ipykernel_1521/4050028980.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw['hour'] = raw.index.hour


In [23]:
aggregated[aggregated.sensor_id == '15845']

,date,hour,sensor_id,temperature_mean,lat,lon
31,2020-01-01,0,15845,10.074,52.498,13.466
